In [1]:
import pandas as pd 
import numpy as np
import os

In [2]:
print(os.getcwd())

c:\Users\sydwi\Desktop\QSML\data\QS4ML-VU-100\final_data


In [30]:
raw_merge_data_dir = os.getcwd() + '\\raw_merge_data'
raw_merge_data_files = ['walk_merge_raw.csv', 'bike_merge_raw.csv', 'run_merge_raw.csv', 'sit_merge_raw.csv', 'syn_merge_raw.csv']

#所有dataframe都存在这里
df_list = []

for i in range(len(raw_merge_data_files)):
    df = pd.read_csv(raw_merge_data_dir + '\\' + raw_merge_data_files[i], low_memory=False)
    df_list.append(df)

<class 'pandas.core.frame.DataFrame'>
